In [1]:
import numpy as np
import tensorflow as tf
import argparse
import os
from six.moves import cPickle
import nltk
import time
import queue as Q
from operator import itemgetter
from model_back import Model as Model_back
from model_forw import Model as Model_forw

import itertools
from numpy.random import choice
from collections import defaultdict
from gensim.models.keyedvectors import KeyedVectors
import string
import re
import collections
import sys
import pickle
import getopt
import random
from collections import defaultdict, Counter
import itertools
counts = defaultdict(Counter)
from Traversal import *
from functions import *

c:\users\hp\anaconda2\envs\translator\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
c:\users\hp\anaconda2\envs\translator\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#LOAD DIRECTORY OF MODELS
text_list = [("data\\frost\\input.txt","save_2"),("data\frost\input.txt","frost_model")]
#np.random.shuffle(text_list)
t = text_list[0][0] #THIS TEXT IS THE VOCAB!
save_dir = text_list[0][1] #THIS IS THE MODEL DIRECTORY
t_back=text_list[1][0]
save_dir_back=text_list[1][1]

width = 20
wordPools = [set([]) for n in range(4)]

In [3]:

# load glove to a gensim model
glove_model = KeyedVectors.load_word2vec_format('./storyline_for_reference/glove.6B.300d.word2vec.txt',binary=False)

# system arguments
topics = [sys.argv[1]]
try:
    seed = int(sys.argv[2])
except:
    seed = 1
    
text = open(t)
text = text.read()

np.random.seed(seed) # seed for reproducibility
words = [simple_clean(word) for word in text.split()]
uniques = set()
for word in words:
    uniques.add(word)
corpus = list(uniques)
dictWordTransitions = {}
glove_words = glove_model.vocab.keys() #SHOULDN'T THIS HAVE BEEN MOST COMMON WORDS IN THE CORPUS?
# CORPUS LIMITED TO WORDS IN GLOVE VOCAB
word_counts = collections.Counter([x for x in words])
corpi = [x[0] for x in word_counts.most_common() if x[0] in glove_words]
dictPartSpeechTags = createPartSpeechTags(corpi)
dictPossiblePartsSpeech = possiblePartsSpeechPaths()
vowels = ["AA", "AE", "AH", "AO", "AW", "AY", "EH", "ER", "EY", "IH", "IY", "OW", "OY", "UH", "UW"]
width = 20
wordPools = [set([]) for n in range(4)]


In [20]:
def genPoem_forward(save_dir,width, wordPools, word_last, TemplatePOS):
    start = time.time()
    def sampleLine(lst, cut):
        ''' samples from top "cut" lines, the distribution being the softmax of true sentence probabilities'''
        probs = list()
        for i in range(cut):
            probs.append(np.exp(lst[i][0][0][0]))
        probs = np.exp(probs) / sum(np.exp(probs))
        index = np.random.choice(cut,1,p=probs)[0]
        return lst[index][1][1]
    tf.reset_default_graph()
    with open(os.path.join(save_dir, 'config.pkl'), 'rb') as f:
        saved_args = cPickle.load(f)
    with open(os.path.join(save_dir, 'words_vocab.pkl'), 'rb') as f:
        word_keys, vocab = cPickle.load(f)
    model = Model_forw(saved_args, True)
    print (len(vocab))
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state(save_dir)
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
            poem = []
            line_num = 0
            wordPool_ind = 0
            state = sess.run(model.initial_state)
            init_score = np.array([[0]])
            lst = search_forward(model, vocab, init_score,[word_last],state, sess, 1,\
                              dictPartSpeechTags,dictPossiblePartsSpeech,width,wordPools[wordPool_ind], PartOfSpeachSet, TemplatePOS)
            lst.sort(key=itemgetter(0), reverse = True)
            # line diagnostics
            #for i in range(10)
            #    print(lst[i][0][0][0], lst[i][1][1])
            #choice = sampleLine(lst, min(10,len(lst)))
            #print (choice)
            #poem.append(choice)
            #line_num+=1
            #if(line_num>3):
            #    line_num = 0
            #    wordPool_ind+=1
        #poem = postProcess(poem,sess,vocab,model)
    print("Generation took {:.3f} seconds".format(time.time() - start))
    return lst





In [21]:
def genPoem_backward(save_dir,width, wordPools, word_last, TemplatePOS):
    start = time.time()
    def sampleLine(lst, cut):
        ''' samples from top "cut" lines, the distribution being the softmax of true sentence probabilities'''
        probs = list()
        for i in range(cut):
            probs.append(np.exp(lst[i][0][0][0]))
        probs = np.exp(probs) / sum(np.exp(probs))
        index = np.random.choice(cut,1,p=probs)[0]
        return lst[index][1][1]
    tf.reset_default_graph()
    with open(os.path.join(save_dir, 'config.pkl'), 'rb') as f:
        saved_args = cPickle.load(f)
    with open(os.path.join(save_dir, 'words_vocab.pkl'), 'rb') as f:
        word_keys, vocab = cPickle.load(f)
    print(len(vocab))
    model = Model_back(saved_args, True)
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state(save_dir)
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
            poem = []
            line_num = 0
            wordPool_ind = 0
            state = sess.run(model.initial_state)
            init_score = np.array([[0]])
            lst = search_back(model, vocab, init_score,[word_last],state, sess, 1,\
                              dictPartSpeechTags,dictPossiblePartsSpeech,width,wordPools[wordPool_ind], PartOfSpeachSet, TemplatePOS)
            lst.sort(key=itemgetter(0), reverse = True)
            # line diagnostics
            #for i in range(10)
            #    print(lst[i][0][0][0], lst[i][1][1])
            #choice = sampleLine(lst, min(10,len(lst)))
            #print (choice)
            #poem.append(choice)
            #line_num+=1
            #if(line_num>3):
            #    line_num = 0
            #    wordPool_ind+=1
        #poem = postProcess(poem,sess,vocab,model)
    print("Generation took {:.3f} seconds".format(time.time() - start))
    return lst

In [108]:
def force_middle(save_dir, width, lst, next_word):
    def decayRepeat(word,sequence, scale):
        safe_repeat_words = []
        #safe_repeat_words = set(["with,the,of,in,i"])
        score_adjust = 0
        decr = -scale
        for w in range(len(sequence)):
            if(word==sequence[w] and word not in safe_repeat_words):
                score_adjust += decr
            decr += scale/10 #decreases penalty as the words keep getting further from the new word
        return score_adjust
    tf.reset_default_graph()
    with open(os.path.join(save_dir, 'config.pkl'), 'rb') as f:
        saved_args = cPickle.load(f)
    with open(os.path.join(save_dir, 'words_vocab.pkl'), 'rb') as f:
        word_keys, vocab = cPickle.load(f)
    model = Model_forw(saved_args, True)
    print (len(vocab))
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state(save_dir)
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
            poem = []
            line_num = 0
            wordPool_ind = 0
            ret=[]
            for tup in lst:
                state_init = tup[1][0][1]
                prob_seq =  tup[1][0][0]
                sequence=tup[1][1]
                scale = .02 #scale is the significant magnitude required to affect the score of bad/good things
                dist, state = model.compute_fx(sess, vocab, prob_seq, sequence, state_init, 1)
                #PREVENTS REPEAT ADJACENT WORDS OR PROBLEM-TAGGED WORDS
                if(next_word == sequence[-1]):
                    continue
                #if(partsOfSpeechFilter(sequence[-1],next_word,dictPartSpeechTags,dictPossiblePartsSpeech)):
                #    continue
                #FACTORS IN SCORE ADJUSTMENTS
                score_adjust = decayRepeat(next_word, sequence, 100*scale) #repeats
                score_adjust += scale*len(next_word)/50 #length word
                #if(next_word in wordPool):
                #    score_adjust += scale
                #CALCULATES ACTUAL SCORE
                key = np.array([[vocab[next_word]]])
                new_prob = dist[key]
                score_tuple = (new_prob, state)
                score_tup = (score_tuple[0]+score_adjust,score_tuple[1]) #NOTE SHOULD SCORE_ADJUST BE ADDED HERE OR JUST IN THE ITEM LINE?
                item = (score_tup[0],(score_tup, sequence+[next_word]))
                if(item[0]==[[-float("inf")]]):
                    continue
                ret+=[item]
            ret.sort(key=itemgetter(0), reverse = True)
            return ret[:width]

In [36]:
#LOAD DICTIONARIES CREATED IN EXTRAXTING_POSTAG.IPYNB
with open('postag_dict.p','rb') as f:
    postag_dict=pickle.load(f)
    
PartOfSpeachSet=postag_dict[1]
TemplatePOS=['PRP', 'VBZ', 'DT', 'NN', 'VBG', 'TO', 'DT', 'NN']

In [90]:
#function to get a template based on two words
def pos_synset(words, pos_dict):
    postag_nn=[]
    print(words)
    for word in words:
        pos_word=postag_dict[2][word][0]
        postag_nn.append(pos_word)
    print (postag_nn)
    pos=str(postag_nn[0])+'-'+str(postag_nn[1])
    if pos in set(list(pos_dict.keys())):
        possible_templates=pos_dict[pos]
        possible=[k_ for k_ in possible_templates if len(k_[0])<8]
    else:
        pos=str(postag_nn[1])+'-'+str(postag_nn[0])
        if pos in set(list(pos_dict.keys())):
            possible_templates=pos_dict[pos]
            possible=[k_ for k_ in possible_templates if len(k_[0])<8]
            #postag_nn=[postag_nn[1], postag_nn[0]]
            words=[words[1], words[0]]
        else:
            return None
    template_list=list(random.choice(possible))
    if postag_nn[0]==postag_nn[1]:
        template_list.append(words)
        return template_list
    else:
        ww=[words[postag_nn.index(postag_nn[0])], words[postag_nn.index(postag_nn[1])]]
        template_list.append(ww)
        return template_list

## combine

In [187]:
def generate_line(word1, word2):
    nouns=[word1, word2]

    postag_nn=[]
    #SELECT A POS SAMPLE
    """for noun in nouns:
        postag_nn.append(nltk.pos_tag([noun])[0][1])
    if postag_nn.count('NN')>1:
        postag=list(random.choice(postag_dict[0]['NN-NN']))
        nouns=nouns
        postag.append(nouns)
    elif postag_nn.count('NN')==1 and postag_nn.count('NNS')==1:
        postag=list(random.choice(postag_dict[0]['NN-NNS']))
        nouns=[nouns[postag_nn.index('NN')], nouns[postag_nn.index('NNS')]]
        postag.append(nouns)
    elif postag_nn.count('NNS')>1:
        postag=list(random.choice(postag_dict[0]['NNS-NNS']))
        nouns=nouns
        postag.append(nouns)
    else:
        print ('WORDS ARE NOT NOUNS')
        #return None
        """
    postag=pos_synset(nouns, postag_dict[0])
    if postag==None:
        return None
    #postag=[['CC', 'PRP', 'RB', 'NN', 'IN', 'DT', 'NN', 'IN', 'NN'], [3,6], ['night', 'moon']]
    print (postag[0])
    last_position=np.argmax(postag[1])
    first_position=np.argmin(postag[1])

    
    last_noun=postag[2][last_position]
    first_noun=postag[2][first_position]
    
    first_pos=postag[1][first_position]
    last_pos=postag[1][last_position]
    template=postag[0]
    print ('First noun: '+ str(first_noun))
    print ('Last noun: '+ str(last_noun))
    print (template[first_pos:last_pos])

    print ('#################################\n')

    list_1= genPoem_forward(save_dir,width,wordPools,first_noun, template[first_pos:last_pos])
    list_2=force_middle(save_dir, width, list_1[:50], last_noun)


    tf.reset_default_graph()
    with open(os.path.join(save_dir, 'config.pkl'), 'rb') as f:
        saved_args = cPickle.load(f)
    with open(os.path.join(save_dir, 'words_vocab.pkl'), 'rb') as f:
        word_keys, vocab = cPickle.load(f)
    model = Model_forw(saved_args, True)
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state(save_dir)
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
            poem = []
            line_num = 0
            wordPool_ind = 0
            tt_3=[]
            for element in list_2:
                seq=element[1][1]
                state = element[1][0][1]
                init_score = element[1][0][0]
                seq=element[1][1]
                lst = search_forward(model, vocab, init_score,seq,state, sess, 1,\
                                  dictPartSpeechTags,dictPossiblePartsSpeech,width,wordPools[wordPool_ind], PartOfSpeachSet, template[first_pos:])
                tt_3+=lst
            tt_3.sort(key=itemgetter(0), reverse = True)
            
    tf.reset_default_graph()
    with open(os.path.join(save_dir_back, 'config.pkl'), 'rb') as f:
        saved_args = cPickle.load(f)
    with open(os.path.join(save_dir_back, 'words_vocab.pkl'), 'rb') as f:
        word_keys, vocab = cPickle.load(f)
    model = Model_back(saved_args, True)
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state(save_dir_back)
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
            poem = []
            line_num = 0
            wordPool_ind = 0
            tt_4=[]
            for tup in tt_3[:30]:
                seq=tup[1][1]
                init_score, state=model.score_a_list(sess, vocab, seq)
                lst = search_back(model, vocab, init_score,seq,state, sess, 1,\
                              dictPartSpeechTags,dictPossiblePartsSpeech,width,wordPools[wordPool_ind], PartOfSpeachSet, template)
                
                tt_4+=lst
    tt_4.sort(key=itemgetter(0), reverse = True)
                
                         
    return template, tt_4

# Results

# one word

In [218]:
TemplatePOS=['PRP', 'VBZ', 'DT', 'NN', 'VBG', 'TO', 'DT', 'NN']

## input first word

In [221]:
poem=genPoem_forward(save_dir, 20, wordPools, 'it', TemplatePOS)

print ('*************\n')
print ('POS: ', TemplatePOS)

print('\n'+' '.join(poem[0][1][1]))

6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
Generation took 9.956 seconds
*************

POS:  ['PRP', 'VBZ', 'DT', 'NN', 'VBG', 'TO', 'DT', 'NN']

it is a man standing to the ground


## input last word

In [222]:
poem=genPoem_backward(save_dir_back, 20, wordPools, 'darkness', TemplatePOS)

print ('*************\n')
print ('POS: ', TemplatePOS)

print('\n'+' '.join(poem[0][1][1]))

6885
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699


C:\Users\hp\Documents\Data_Science\model_back.py:207: RuntimeWarning: divide by zero encountered in log
  dist = p.squeeze() + np.log(probs.squeeze())


Generation took 5.320 seconds
*************

POS:  ['PRP', 'VBZ', 'DT', 'NN', 'VBG', 'TO', 'DT', 'NN']

it is a scythe listening to the darkness


In [227]:
poem=genPoem_backward(save_dir_back, 20, wordPools, 'human', TemplatePOS)

print ('*************\n')
print ('POS: ', TemplatePOS)

print('\n'+' '.join(poem[0][1][1]))

6885
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
Generation took 5.217 seconds
*************

POS:  ['PRP', 'VBZ', 'DT', 'NN', 'VBG', 'TO', 'DT', 'NN']

he makes a spirit listening to that human


In [236]:
TemplatePOS=['PRP', 'VBP', 'DT', 'NN', 'CC', 'VBD', 'PRP', 'RP', 'JJ', 'NNS']
poem=genPoem_backward(save_dir_back, 20, wordPools, 'people', TemplatePOS)

print ('*************\n')
print ('POS: ', TemplatePOS)

print('\n'+' '.join(poem[0][1][1]))

6885
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699


C:\Users\hp\Documents\Data_Science\model_back.py:207: RuntimeWarning: divide by zero encountered in log
  dist = p.squeeze() + np.log(probs.squeeze())


Generation took 5.815 seconds
*************

POS:  ['PRP', 'VBP', 'DT', 'NN', 'CC', 'VBD', 'PRP', 'RP', 'JJ', 'NNS']

you know the resin and put it down twenty people


# Evaluate different templates

### memories and old

In [180]:
import pandas as pd
temp_score=[]
for x in range(6):
    try:
        template, k=generate_line('memories','old')
    except KeyError:
        continue
    if k!=[]:
        k=k[0]
        #print (k[0][0][0])
        row=(' '.join(k[1][1]),k[0][0]/len(k[1][1]), template) 
        temp_score.append(row)
pd.DataFrame(temp_score, columns=['line', 'score', 'POS'])

['memories', 'old']
['NNS', 'JJ']
['DT', 'JJ', 'NN', 'VBN', 'IN', 'IN', 'NNS']
First noun: old
Last noun: memories
['JJ', 'NN', 'VBN', 'IN', 'IN']
#################################

6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
Generation took 8.079 seconds
6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
6885
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
['memories', 'old']
['NNS', 'JJ']
['IN', 'DT', 'NN', 'IN', 'CD', 'JJ', 'NNS']
First noun: old
Last noun: memories
['JJ']
#################################

6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
Generation took 7.497 seconds
6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
6885
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
['memories', 'old']
['

C:\Users\hp\Documents\Data_Science\model_back.py:207: RuntimeWarning: divide by zero encountered in log
  dist = p.squeeze() + np.log(probs.squeeze())


['memories', 'old']
['NNS', 'JJ']
['DT', 'NN', 'IN', 'JJ', 'NNP', 'NNS']
First noun: old
Last noun: memories
['JJ', 'NNP']
#################################

6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
Generation took 6.442 seconds
6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
6885
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
['memories', 'old']
['NNS', 'JJ']
['NNP', 'VBP', 'RB', 'JJ', 'NN', 'NNS', 'VBP']
First noun: old
Last noun: memories
['JJ', 'NN']
#################################

6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
Generation took 11.757 seconds
6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
6885
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
['memories', 'old']
['NNS', 'JJ']
['

KeyboardInterrupt: 

In [181]:
pd.DataFrame(temp_score, columns=['line', 'score', 'POS'])

,line,score,POS
0,the old way come out beside memories,[-7.700625],"[DT, JJ, NN, VBN, IN, IN, NNS]"
1,in the bed of one old memories,[-5.896367],"[IN, DT, NN, IN, CD, JJ, NNS]"
2,is the old memories of man while,[-8.425577],"[VBZ, DT, JJ, NNS, IN, NN, NN]"
3,the night of old christmas memories,[-5.3155246],"[DT, NN, IN, JJ, NNP, NNS]"
4,dont know so old thousand memories are,[-6.8256645],"[NNP, VBP, RB, JJ, NN, NNS, VBP]"


### bring and light

In [203]:
import pandas as pd
temp_score=[]
for x in range(8):
    try:
        template, k=generate_line('bring','light')
    except KeyError:
        continue
    if k!=[]:
        k=k[0]
        #print (k[0][0][0])
        row=(' '.join(k[1][1]),k[0]/len(k[1][1]), template) 
        temp_score.append(row)
print (' ...')

['bring', 'light']
['VB', 'NN']
['CC', 'VB', 'DT', 'NNS', 'NN']
First noun: bring
Last noun: light
['VB', 'DT', 'NNS']
#################################

6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
Generation took 6.344 seconds
6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
['bring', 'light']
['VB', 'NN']
['CC', 'DT', 'NN', 'NNS', 'VB', 'RB']
First noun: light
Last noun: bring
['NN', 'NNS']
#################################

6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
Generation took 6.147 seconds
6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
['bring', 'light']
['VB', 'NN']
['PRP$', 'NNS', 'IN', 'DT', 'NN', 'TO', 

C:\Users\hp\Documents\Data_Science\model_back.py:207: RuntimeWarning: divide by zero encountered in log
  dist = p.squeeze() + np.log(probs.squeeze())


['bring', 'light']
['VB', 'NN']
['PRP', 'VBZ', 'VB', 'DT', 'NNS', 'VBP', 'NN']
First noun: bring
Last noun: light
['VB', 'DT', 'NNS', 'VBP']
#################################

6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
Generation took 6.857 seconds
6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
['bring', 'light']
['VB', 'NN']
['PRP$', 'NN', 'NNS', 'MD', 'RB', 'VB', 'IN']
First noun: light
Last noun: bring
['NN', 'NNS', 'MD', 'RB']
#################################

6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
Generation took 6.059 seconds
6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
['bring', 'light']
['VB', 'NN'

In [204]:
pd.DataFrame(temp_score, columns=['line', 'score', 'POS'])

,line,score,POS
0,and bring the feet light,[[-5.5792217]],"[CC, VB, DT, NNS, NN]"
1,and the light feet bring down,[[-7.085303]],"[CC, DT, NN, NNS, VB, RB]"
2,their places among the light to bring,[[-5.4011836]],"[PRP$, NNS, IN, DT, NN, TO, VB]"
3,it is bring no people are light,[[-6.790763]],"[PRP, VBZ, VB, DT, NNS, VBP, NN]"
4,your light people may not bring out,[[-8.001394]],"[PRP$, NN, NNS, MD, RB, VB, IN]"
5,a light in something to bring things,[[-3.7749782]],"[DT, NN, IN, VBG, TO, VB, NNS]"
6,the light people never do to bring,[[-6.738369]],"[DT, NN, NNS, RB, VBP, TO, VB]"
7,the light with trees will bring,[[-6.1556816]],"[DT, NN, IN, NNS, MD, VB]"


### climb stairs

In [205]:
import pandas as pd
temp_score=[]
for x in range(8):
    try:
        template, k=generate_line('climb','stairs')
    except KeyError:
        continue
    if k!=[]:
        k=k[0]
        #print (k[0][0][0])
        row=(' '.join(k[1][1]),k[0]/len(k[1][1]), template) 
        temp_score.append(row)
print (' ...')

['climb', 'stairs']
['VB', 'NNS']
['CC', 'DT', 'NN', 'NNS', 'VB', 'RB']
First noun: stairs
Last noun: climb
['NNS']
#################################

6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
Generation took 5.905 seconds
6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699


C:\Users\hp\Documents\Data_Science\model_back.py:207: RuntimeWarning: divide by zero encountered in log
  dist = p.squeeze() + np.log(probs.squeeze())


['climb', 'stairs']
['VB', 'NNS']
['VB', 'DT', 'NN', 'NNS', 'VBP']
First noun: climb
Last noun: stairs
['VB', 'DT', 'NN']
#################################

6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
Generation took 6.885 seconds
6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
['climb', 'stairs']
['VB', 'NNS']
['TO', 'VB', 'NNS', 'DT', 'JJ', 'NN']
First noun: climb
Last noun: stairs
['VB']
#################################

6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
Generation took 5.583 seconds
6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
['climb', 'stairs']
['VB', 'NNS']
['VB', 'DT', 'NN', 'NNS', 'VBP']
First 

In [206]:
pd.DataFrame(temp_score, columns=['line', 'score', 'POS'])

,line,score,POS
0,and the bright stairs climb down,[[-10.348529]],"[CC, DT, NN, NNS, VB, RB]"
1,climb the mountain stairs have,-10.874,"[VB, DT, NN, NNS, VBP]"
2,to climb stairs the open sky,[[-9.5463915]],"[TO, VB, NNS, DT, JJ, NN]"
3,climb the mountain stairs have,-10.874,"[VB, DT, NN, NNS, VBP]"
4,climb the mountain stairs have,-10.874,"[VB, DT, NN, NNS, VBP]"
5,like stairs were dead to climb climbing,[[-7.734122]],"[IN, NNS, VBD, JJ, TO, VB, NN]"
6,to climb in whats winter and stairs,[[-5.315476]],"[TO, VB, IN, IN, NN, CC, NNS]"


##  eyes emotion

In [215]:
import pandas as pd
temp_score=[]
for x in range(8):
    try:
        template, k=generate_line('emotion','eyes')
    except KeyError:
        continue
    if k!=[]:
        k=k[0]
        #print (k[0][0][0])
        row=(' '.join(k[1][1]),k[0]/len(k[1][1]), template) 
        temp_score.append(row)
print (' ...')

['emotion', 'eyes']
['NN', 'NNS']
['CC', 'VBD', 'JJ', 'NNS', 'IN', 'NN']
First noun: eyes
Last noun: emotion
['NNS', 'IN']
#################################

6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
Generation took 5.864 seconds
6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699


C:\Users\hp\Documents\Data_Science\model_back.py:207: RuntimeWarning: divide by zero encountered in log
  dist = p.squeeze() + np.log(probs.squeeze())


['emotion', 'eyes']
['NN', 'NNS']
['IN', 'JJ', 'JJ', 'NN', 'NNS', ',']
First noun: emotion
Last noun: eyes
['NN']
#################################

6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
Generation took 6.117 seconds
6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
['emotion', 'eyes']
['NN', 'NNS']
['TO', 'VB', 'NN', 'PRP', 'VBP', 'DT', 'NNS']
First noun: emotion
Last noun: eyes
['NN', 'PRP', 'VBP', 'DT']
#################################

6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
Generation took 5.914 seconds
6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
['emotion', 'eyes']
['NN', 'NNS']
['DT', 'NNS', 'RB', 'VBP', 'NN']
First noun: eyes
Last noun: emotion
['NNS', 'RB', 'VBP']
#

In [216]:
pd.DataFrame(temp_score, columns=['line', 'score', 'POS'])

,line,score,POS
0,and was only eyes of emotion,[[-6.786516]],"[CC, VBD, JJ, NNS, IN, NN]"
1,the eyes then are emotion,[[-10.215985]],"[DT, NNS, RB, VBP, NN]"
2,the eyes not are that emotion,[[-8.730015]],"[DT, NNS, RB, VBP, DT, NN]"
3,eyes and emotion,-9.32077,"[NNS, CC, NN]"
4,other eyes as long as emotion,[[-8.266255]],"[JJ, NNS, IN, RB, IN, NN]"


In [242]:
import pandas as pd
temp_score=[]
for x in range(8):
    try:
        template, k=generate_line('time', 'sound')
    except KeyError:
        continue
    if k!=[]:
        k=k[0]
        #print (k[0][0][0])
        row=(' '.join(k[1][1]),k[0]/len(k[1][1]), template) 
        temp_score.append(row)
print (' ...')

['time', 'sound']
['NN', 'NN']
['IN', 'IN', 'NN', 'CC', 'NN']
First noun: time
Last noun: sound
['NN', 'CC']
#################################

6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
Generation took 10.165 seconds
6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
['time', 'sound']
['NN', 'NN']
['DT', 'NN', 'NN']
First noun: time
Last noun: sound
['NN']
#################################

6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
Generation took 5.927 seconds
6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
['time', 'sound']
['NN', 'NN']
['PRP$', 'NN', 'VBZ', 'NN']
First noun: sound
Last noun: time
['NN', 'VBZ']
##

C:\Users\hp\Documents\Data_Science\model_back.py:207: RuntimeWarning: divide by zero encountered in log
  dist = p.squeeze() + np.log(probs.squeeze())


['time', 'sound']
['NN', 'NN']
['CC', 'IN', 'NN', 'VBG', 'PDT', 'DT', 'NN']
First noun: sound
Last noun: time
['NN', 'VBG', 'PDT', 'DT']
#################################

6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
Generation took 5.888 seconds
6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
['time', 'sound']
['NN', 'NN']
['NN', 'RB', ',', 'CC', 'IN', 'IN', 'NN']
First noun: sound
Last noun: time
['NN', 'RB', ',', 'CC', 'IN', 'IN']
#################################

6885
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
 ...


In [243]:
pd.DataFrame(temp_score, columns=['line', 'score', 'POS'])

,line,score,POS
0,beside that time and sound,[[-4.153561]],"[IN, IN, NN, CC, NN]"
1,the time sound,[[-4.6519017]],"[DT, NN, NN]"
2,her sound is time,[[-4.3387413]],"[PRP$, NN, VBZ, NN]"
3,things to the time sound,[[-4.418531]],"[NNS, TO, DT, NN, NN]"
4,or time in sound taking new hampshire,[[-7.205396]],"[CC, NN, IN, NN, VBG, NNP, NNP]"
5,the long time then made that sound,[[-4.7765946]],"[DT, JJ, NN, RB, VBN, IN, NN]"
6,and whats sound standing half the time,[[-5.2065086]],"[CC, IN, NN, VBG, PDT, DT, NN]"
